# Regresión lineal para charn

## Df clientes

In [15]:
import pyodbc
import pandas as pd
import logging

# Configuración de logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# Configuración de conexión a SQL Server LOCAL
LOCAL_SERVER = "localhost"
LOCAL_DATABASE = "master"
LOCAL_DRIVER = "{ODBC Driver 17 for SQL Server}"

local_conn_str = f"DRIVER={LOCAL_DRIVER};SERVER={LOCAL_SERVER};DATABASE={LOCAL_DATABASE};Trusted_Connection=yes;TrustServerCertificate=yes"

# Ruta a la carpeta con los archivos SQL
SQL_FOLDER_PATH = "src/sql/customer/"
SQL_FILE = "SQLQuery_Customer.sql"

# Leer la consulta desde el archivo SQL
with open(f"{SQL_FOLDER_PATH}{SQL_FILE}", "r", encoding="iso-8859-1") as file:
    query = file.read()

# Conectar a SQL Server y ejecutar la consulta
try:
    conn = pyodbc.connect(local_conn_str)
    df_customer = pd.read_sql(query, conn)
    conn.close()
    
    # Mostrar las primeras filas
    print(df_customer.head())
    
    logging.info("Consulta ejecutada exitosamente y convertida a DataFrame.")

except Exception as e:
    logging.error(f"Error al ejecutar la consulta: {e}")

C:\Users\aleja\AppData\Local\Temp\ipykernel_8472\2935947387.py:26: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_customer = pd.read_sql(query, conn)
2025-03-24 15:09:33,115 - INFO - Consulta ejecutada exitosamente y convertida a DataFrame.


   Customer_ID  Edad Fecha_nacimiento GENERO       CP          poblacion  \
0          664    31       07/01/1993      F  28850.0  Torrejón de Ardoz   
1          672    53       13/01/1971      M   8320.0          El Masnou   
2          712    30       07/01/1994      M  36940.0             Cangas   
3          741    34       08/01/1990      F  48960.0           Galdakao   
4          741    34       08/01/1990      F  48960.0           Galdakao   

    provincia STATUS_SOCIAL  RENTA_MEDIA_ESTIMADA  \
0      Madrid             A                 30957   
1   Barcelona             B                 27369   
2  Pontevedra             I                 24590   
3     Bizkaia            NA                     0   
4     Bizkaia            NA                     0   

   ENCUESTA_ZONA_CLIENTE_VENTA  ...  Logistic_date   Prod_date  \
0                           89  ...              0           0   
1                           99  ...              0           0   
2                         

## Regresión lineal

### Preparación de los datos

In [19]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm

# 1. Agrupar todas las variables EXCEPTO PVP (que mantendremos a nivel transacción)
df_customer_agrupado = df_customer.groupby('Customer_ID').agg({
    'Margen_eur': 'mean',        # Margen agrupado
    'Car_Age': 'mean',           # Edad coche agrupada
    'Edad': 'mean',              # Edad cliente agrupada
    'Km_medio_por_revision': 'mean',  # Km medios agrupados
    'Margen_por_unidad': 'mean', # Margen por unidad agrupado
    'CHARN': 'max'               # CHARN (variable objetivo)
}).reset_index()

# 2. Merge con los valores ORIGINALES de PVP (sin agrupar)
df_customer_final = pd.merge(
    df_customer[['Customer_ID', 'PVP']],  # PVP desagrupado (mantiene valores individuales)
    df_customer_agrupado,
    on='Customer_ID',
    how='right'
)

# 3. Limpieza datos
df_customer_final = df_customer_final.dropna()
df_customer_final = df_customer_final.replace([np.inf, -np.inf], np.nan).dropna()

ValueError: Grouper for 'Customer_ID' not 1-dimensional

In [20]:
# Verificar las columnas disponibles
print("Columnas en df_customer:", df_customer.columns.tolist())

# Verificar si Customer_ID existe y es única
if 'Customer_ID' in df_customer.columns:
    print("\nValores únicos de Customer_ID:", df_customer['Customer_ID'].nunique())
    print("Tipo de datos:", df_customer['Customer_ID'].dtype)
    
    # Verificar si hay valores duplicados
    print("\nFilas duplicadas por Customer_ID:", df_customer.duplicated(subset=['Customer_ID']).sum())
else:
    print("Error: La columna 'Customer_ID' no existe en el DataFrame")

Columnas en df_customer: ['Customer_ID', 'Edad', 'Fecha_nacimiento', 'GENERO', 'CP', 'poblacion', 'provincia', 'STATUS_SOCIAL', 'RENTA_MEDIA_ESTIMADA', 'ENCUESTA_ZONA_CLIENTE_VENTA', 'ENCUESTA_CLIENTE_ZONA_TALLER', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'U2', 'Max_Mosaic_G', 'Max_Mosaic2', 'Renta_Media', 'F2', 'Mosaic_number', 'CODE', 'PVP', 'MANTENIMIENTO_GRATUITO', 'SEGURO_BATERIA_LARGO_PLAZO', 'FIN_GARANTIA', 'COSTE_VENTA_NO_IMPUESTOS', 'IMPUESTOS', 'EXTENSION_GARANTIA', 'EN_GARANTIA', 'Customer_ID', 'DATE_UTIMA_REV', 'DIAS_DESDE_ULTIMA_REVISION', 'Km_medio_por_revision', 'km_ultima_revision', 'Revisiones', 'DIAS_DESDE_LA_ULTIMA_ENTRADA_TALLER', 'DIAS_EN_TALLER', 'QUEJA', 'MOTIVO_VENTA_ID', 'MOTIVO_VENTA', 'FORMA_PAGO_ID', 'FORMA_PAGO', 'FORMA_PAGO_GRUPO', 'Fue_Lead', 'Lead_compra', 't_logist_days', 't_prod_date', 't_stock_dates', 'Logistic_date', 'Prod_date', 'Origen_Compra_ID', 'Origen', 'Car_Age', 'Margen_eur_bruto', 'Margen_eur', 'CHARN', 'Margen_por_unidad', 'le

AttributeError: 'DataFrame' object has no attribute 'dtype'

### Análisis exploratorio

In [ ]:
# Matriz de correlación
corr_matrix = df_customer_final.corr()
print(corr_matrix['CHARN'].sort_values(ascending=False))

# Visualización de relaciones
pd.plotting.scatter_matrix(df_customer_final, figsize=(12, 8))
plt.show()

### Preparación del modelo

In [ ]:
# Definir variables independientes y dependiente
X = df_customer_final[['PVP', 'Margen_eur', 'Car_Age', 'Edad', 'Km_medio_por_revision', 'Margen_por_unidad']]
y = df_customer_final['CHARN']

# Añadir constante para el intercepto
X = sm.add_constant(X)

# Dividir en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Entrenamiento del modelo

In [ ]:
# Modelo con statsmodels (para análisis detallado)
model_sm = sm.OLS(y_train, X_train).fit()
print(model_sm.summary())

# Modelo con scikit-learn (para predicciones)
model = LinearRegression()
model.fit(X_train.drop('const', axis=1), y_train)

### Evaluación del modelo

In [ ]:
# Predecir en el conjunto de prueba
y_pred = model.predict(X_test.drop('const', axis=1))

# Métricas de evaluación
print("R²:", r2_score(y_test, y_pred))
print("MSE:", mean_squared_error(y_test, y_pred))

# Visualización de predicciones vs reales
plt.scatter(y_test, y_pred)
plt.xlabel("Valores Reales")
plt.ylabel("Predicciones")
plt.title("Predicciones vs Valores Reales")
plt.show()